In [ ]:
import os
import pandas as pd
import tensorflow as tf
import numpy as np
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D, Flatten, BatchNormalization, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import time

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Path to the directory containing PNG images
image_dir = '/content/drive/My Drive/Semester 5/Pemrosesan Suara/project/Hasil GFCC'

# Load data from CSV
csv_filename = '/content/drive/My Drive/Semester 5/Pemrosesan Suara/project/Hasil GFCC/labels_and_file_paths.csv'
data = pd.read_csv(csv_filename)

# Define label mapping
label_to_index = {
    'go': 0,
    'stop': 1,
    'left': 2,
    'right': 3,
}

# Map labels to numeric values
data['Label'] = data['Label'].map(label_to_index)

# Load and preprocess images
images = []  # To store preprocessed images
labels = []  # To store labels

def preprocess_image(image_path):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
    img = tf.keras.preprocessing.image.img_to_array(img)
    img = tf.keras.applications.resnet_v2.preprocess_input(img)
    return img

for filename in data['FilePath']:
    image_path = os.path.join(image_dir, filename)
    image = preprocess_image(image_path)
    images.append(image)
    labels.append(data['Label'])

# Convert labels to one-hot encoding
y_encoded = to_categorical(data['Label'], num_classes=len(label_to_index))

# Split data into training and testing sets
X_train, X_val, y_train_encoded, y_val_encoded = train_test_split(images, y_encoded, test_size=0.3, random_state=42)

# Create a ResNet50V2 model
base_model = ResNet50V2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add Dense layers for classification
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Additional layers - Set 1
x = BatchNormalization()(x)
x = Dense(64, activation='relu')(x)

# Additional layers - Set 2
x = BatchNormalization()(x)
x = Dropout(rate=0.5)(x)
x = Dense(128, activation='relu')(x)

# Additional layers - Set 3
x = BatchNormalization()(x)
x = Dropout(rate=0.5)(x)
x = Dense(256, activation='relu')(x)

predictions = Dense(len(label_to_index), activation='softmax')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

datagen.fit(X_train)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=4)

# Train the model
start_time = time.time()

history = model.fit(
    datagen.flow(np.array(X_train), y_train_encoded, batch_size=32),
    validation_data=(np.array(X_val), y_val_encoded),
    steps_per_epoch=len(X_train) / 32,
    epochs = 50,
    callbacks=[early_stopping]
)

end_time = time.time()
execution_time = end_time - start_time

# Evaluate the model
y_pred = model.predict(np.array(X_val))
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_val_encoded, axis=1)

# Calculate and print accuracy, precision, and F1 score for each class
class_names = label_to_index.keys()
classification_rep = classification_report(y_true, y_pred_classes, target_names=class_names)
print("Classification Report:\n", classification_rep)

# Create and print the confusion matrix
confusion = confusion_matrix(y_true, y_pred_classes)
print("Confusion Matrix:\n", confusion)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
plt.imshow(confusion, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.colorbar()

classes = label_to_index.keys()
tick_marks = np.arange(len(classes))
plt.xticks(tick_marks, classes, rotation=45)
plt.yticks(tick_marks, classes)

plt.xlabel('Predicted Label')
plt.ylabel('True Label')

# Add text annotations
for i in range(len(classes)):
    for j in range(len(classes)):
        plt.text(j, i, format(confusion[i, j], 'd'),
                 horizontalalignment="center",
                 color="white" if confusion[i, j] > confusion.max() / 2 else "black")

plt.show()

# Print execution time
print(f"Execution time: {execution_time:.2f} seconds")
